In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.relativedelta import relativedelta

In [ ]:
date_of_birth = pd.datetime(1994,1,18)
height = 186
sex = 'm'
exercise_level = 2

In [ ]:
df_calories = pd.read_csv('calories.csv', parse_dates=['ts'])

In [ ]:
df_calories.tail()

In [ ]:
df_weight = pd.read_csv('weight.csv', parse_dates=['ts'])
df_weight['weight_avg'] = pd.np.average(df_weight[['weight_am', 'weight_pm']], axis=1)
df_weight = df_weight.set_index('ts')

In [ ]:
df_weight.tail()

In [ ]:
df_calories_daily = df_calories.groupby(pd.Grouper(freq='D', key='ts')).agg({'calories': 'sum'}).reset_index()
df_calories_daily['age'] = df_calories_daily.apply(lambda row: relativedelta(row.ts, date_of_birth).years, axis=1)
df_calories_daily = df_calories_daily.set_index('ts')
df_calories_daily.tail()

In [ ]:
df_calories_weight = df_calories_daily
df_calories_weight['weight'] = df_weight['weight_avg']
df_calories_weight['bodyfat'] = df_weight['bodyfat']

In [ ]:
def daily_maintenance_rate(age,height,weight,sex,bodyfat,exercise_level): 
    exercise_factor = (1.2+exercise_level*0.175)
    
    if bodyfat:
        return 370+(21.6*(weight*1-bodyfat/100))*exercise_factor
    
    s = 5
    if sex != "m": s = -161
    
    mifflin = 10*weight+6.25*height-5*age + s
    
    return mifflin*exercise_factor

In [ ]:
df_calories_weight['tdee'] = df_calories_weight.apply(lambda row: daily_maintenance_rate(row.age,height,row.weight,sex,row.bodyfat,exercise_level), axis=1)

In [ ]:
df_calories_weight['deficit'] = df_calories_weight['tdee'] - df_calories_weight['calories']
df_calories_weight['deficit_cumsum'] = df_calories_weight['deficit'].cumsum()

df_calories_weight['planned_deficit'] = 500
df_calories_weight['planned_deficit_cumsum'] = df_calories_weight['planned_deficit'].cumsum()

df_calories_weight['deficit_mean'] = df_calories_weight['deficit'].expanding().mean()

In [ ]:
df_calories_weight[['planned_deficit', 'deficit', 'deficit_mean']].plot(grid=True,figsize=(16,8))
plt.show()

In [ ]:
df_calories_weight[['planned_deficit_cumsum', 'deficit_cumsum']].plot(grid=True,figsize=(16,8))
plt.show()

In [ ]:
df_calories_weight['expected_weight'] = df_calories_weight['weight'].iloc[0] - df_calories_weight['deficit_cumsum']/3500*0.45

In [ ]:
df_calories_weight[['expected_weight', 'weight']].plot(grid=True,figsize=(16,8))
plt.show()